# Kjøre modellen lokalt
### Laste inn modellen fra Hugging Face

Whisper predicts the language of the source audio automatically. If the source audio language is known a-priori, it can be passed as an argument to the pipeline:

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)



/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
# Sample from the dataset, works
dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

# result = pipe("songs/Glory_Glory_Man_United.mp3")
result = pipe("sound/Glory_Glory_Man_United.mp3", generate_kwargs={"language": "english"})
# result = pipe(sample, generate_kwargs={"language": "english"})
# result = pipe(sample, generate_kwargs={"task": "translate"})

print(result["text"])


 Glory, glory, Man United! Glory, glory, Man United! As the Reds go marching, oh, oh, oh! Just like the buzz we've heard today's done by, we'll keep the red flags flying high. You're gonna see us go from far and wide You're gonna hear them as they sing with pride United, Man United We're the boys in red and we're on our way to Wembley, Wembley We're the famous man united and we're going to Wembley Wembley, Wembley We're the famous man united and we're going to Wembley In 77 it was Doherty Hackinson will make it 83 And everyone will know just who we are They'll be singing K-Sara-Sara United Man United We're the boys in red And we're on our way to Wembley Wembley, Wembley We're the famous band united and we're going to Wembley Wembley, Wembley We're the famous band united and we're going to win for me Glory, glory, Grand United Glory, glory, Grand United Glory, glory, Grand United As the Reds go marching on, on, on Glory, glory, Grand United on, on, Man United! Glory, glory, Man United! 

Glory, glory, Man United! Glory, glory, Man United! As the Reds go marching, oh, oh, oh! Just like the buzz we've heard today's done by, we'll keep the red flags flying high. You're gonna see us go from far and wide You're gonna hear them as they sing with pride United, Man United We're the boys in red and we're on our way to Wembley, Wembley We're the famous man united and we're going to Wembley Wembley, Wembley We're the famous man united and we're going to Wembley In 77 it was Doherty Hackinson will make it 83 And everyone will know just who we are They'll be singing K-Sara-Sara United Man United We're the boys in red And we're on our way to Wembley Wembley, Wembley We're the famous band united and we're going to Wembley Wembley, Wembley We're the famous band united and we're going to win for me Glory, glory, Grand United Glory, glory, Grand United Glory, glory, Grand United As the Reds go marching on, on, on Glory, glory, Grand United on, on, Man United! Glory, glory, Man United! As the Reds go marching on, on, on!

Sliter med en norsk klassiker der ordene ikke høres så tydelig. 

In [10]:
result = pipe("sound/Fredrikstadgutter_våre_veier.mp3", generate_kwargs={"language": "norwegian"})
# result = pipe(sample, generate_kwargs={"language": "english"})
# result = pipe(sample, generate_kwargs={"task": "translate"})

print(result["text"])




Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


 Nørvide, Fremtrikant og Sprit og Kruppensland For fagene de alle har så kjær Vår takke er til Måneberg, Tildensland At vi vel sliter på våre stille bær Fredrikstad, Gudde, våre veier går igjen, og kan ikke seie på vårt land. Må det være der, og kjempe fremme, ut på byen eller hjemmeby. Og så, for gruppe og by, for land i strid, vi kjemper frem til seier på by. Takk for at du så på! Vi venter alltid til gruppens eier, venter hjem i dag. Når blir det en fremdelen katt og strid på gruppens navn? Hva tar vi med? Vi alle har så kjær. Hvor tok en erken av deg vær en kuldenskall. Ha tid og fri på våre sildre vær. Rødde seg gutter våre veier går igjen og kan ikke seie på vårt land. Målet vårt er å knutte frem med unge på byen eller hjemme. Målet vårt er å knytte frem med unge på byen eller hjemmeby. Vi går opp i start, får kutt på byen, får laget fri. Vi kjemper frem til seier på ny. Hvor god det er å bli, jeg knytter nok. Her til topp! Nå er det tilkomm Fredrikstad gutter byen venter Landet 

### Klarer kongens tale meget bra

In [4]:
result = pipe("sound/Norwegian_King's_speech.mp3", generate_kwargs={"language": "norwegian"})
# result = pipe(sample, generate_kwargs={"language": "english"})
# result = pipe(sample, generate_kwargs={"task": "translate"})

print(result["text"])




 Nordmenn er nordlendinger, trøndere, sørlendinger og folk fra alle andre regioner. Nordmenn er også innvandret fra Afghanistan, Pakistan og Polen, Sverige, Somalia og Syria. Det er ikke alltid så lett å si hvor vi er fra, hvilken nasjonalitet vi tilhører. Det vi kaller hjem er der hjertet vårt er, og det kan ikke alltid plasseres innenfor landegrenser. Nordmenn er jenter som er glad i jenter, gutter som er glad i gutter, og jenter og gutter som er glad i hverandre. Nordmenn tror på Gud, Allah, altet og ingenting. Nordmenn liker Grieg, Hugo, Helbilis og Kari Bremnes. Med andre ord, Norge er dere. Norge er oss. Mitt største håp for Norge er at vi skal klare å ta vare på hverandre. At vi skal bygge dette landet videre på tillid, fellesskap og raushet.


In [9]:
# result = pipe("songs/Norwegian_King's_speech.mp3",  generate_kwargs={"task": "translate"})
# result = pipe("songs/Norwegian_King's_speech.mp3",  generate_kwargs={"language": "english"})
result = pipe("sound/Norwegian_King's_speech.mp3",  generate_kwargs={"language": "english", "task": "translate"}, return_timestamps=True)


# result = pipe(sample, generate_kwargs={"language": "english"})
# result = pipe(sample, generate_kwargs={"task": "translate"})
# generate_kwargs={"language": "english"},

print(result["text"])

# return_timestamps=True)


 The Norwegians are northerners, trundlers, southerners and people from all other regions. The Norwegians are also immigrants from Afghanistan, Pakistan, Poland, Sweden, Somalia and Syria. The The Norwegians are girls who are fond of girls, boys who are fond of boys, and girls and boys who are fond of each other. The Norwegians believe in God, Allah, everything and nothing. The Norwegians like Grieg, Hugo, Helbylis and Kari Bremnes. In other words, Norway is you. Norway is us. My biggest hope for Norway is that we will be able to take care of each other. That we will continue to build this country on trust, community and generosity.


In [10]:
print(result)

{'text': ' The Norwegians are northerners, trundlers, southerners and people from all other regions. The Norwegians are also immigrants from Afghanistan, Pakistan, Poland, Sweden, Somalia and Syria. The The Norwegians are girls who are fond of girls, boys who are fond of boys, and girls and boys who are fond of each other. The Norwegians believe in God, Allah, everything and nothing. The Norwegians like Grieg, Hugo, Helbylis and Kari Bremnes. In other words, Norway is you. Norway is us. My biggest hope for Norway is that we will be able to take care of each other. That we will continue to build this country on trust, community and generosity.', 'chunks': [{'timestamp': (0.0, 8.0), 'text': ' The Norwegians are northerners, trundlers, southerners and people from all other regions.'}, {'timestamp': (8.0, 17.0), 'text': ' The Norwegians are also immigrants from Afghanistan, Pakistan, Poland, Sweden, Somalia and Syria.'}, {'timestamp': (49.0, 55.0), 'text': ' The The Norwegians are girls wh

From YT: Norwegians come from the north of Norway, from the middle, -
0:04
- from the south and all the other regions.
0:09
Norwegians are also immigrants from Afghanistan, Pakistan, Poland, -
0:14
- Sweden, Somalia and Syria.
0:25
It is not always easy to say where we come from.
0:28
To which nationality we belong.
0:32
Home is where the heart is.
0:36
That cannot always be placed within country borders.
0:42
Norwegians are girls who love girls.
0:44
Boys who love boys.
0:47
And boys and girls who love each other.
0:51
Norwegians believe in God, Allah, everything and nothing.
0:57
Norwegians like Grieg, Kygo, Hellbillies (Norwegian country rock band) –
1:01
- and Kari Bremnes (Norwegian singer).
1:04
In other words: You are Norway.
1:07
We are Norway.
1:11
My biggest hope for Norway is that we will manage to take care of each other.
1:18
That we can build this country further on trust, sodality and generosity.

Teset om vi kan få talen på nyorsk

In [12]:
# result = pipe("songs/Norwegian_King's_speech.mp3",  generate_kwargs={"language": "nynorsk", "task": "translate"}, return_timestamps=True)
# -> Gir engelsk versjon

result = pipe("sound/Norwegian_King's_speech.mp3",  generate_kwargs={"language": "nynorsk"}, return_timestamps=True)
print(result["text"])

# return_timestamps=True)


 Nordmenn er nordlendinger, trøndere, sørlendinger og folk fra alle andre regioner. Nordmenn er også innvandret fra Afghanistan, Pakistan og Polen, Sverige, Somalia og Syria. Det er ikkje alltid så lett å si hvor vi er ifra, hvilken nasjonalitet vi tilhører. Det vi kallar hjem er der hjertet vårt er, og det kan ikkje alltid plasseres innenfor landegrenser. Nordmenn er jenter som er glad i jenter, gutter som er glad i gutter, og jenter, gutter som er glad i gutter, og jenter og gutter som er glad i hverandre. Nordmenn tror på Gud, Allah, altet og ingenting. Nordmenn liker Grieg, Hugo, Helbilis og Kari Bremnes. Med andre ord, Norge er dere. Norge er oss. Mitt største håp for Norge er at vi skal klare å ta vare på hverandre. At vi skal bygge dette landet videre på tillit, fellesskap og raushet.


## Nynorsk: 

 ### Nordmenn er nordlendinger, trøndere, sørlendinger og folk fra alle andre regioner. Nordmenn er også innvandret fra Afghanistan, Pakistan og Polen, Sverige, Somalia og Syria. Det er ikkje alltid så lett å si hvor vi er ifra, hvilken nasjonalitet vi tilhører. Det vi kallar hjem er der hjertet vårt er, og det kan ikkje alltid plasseres innenfor landegrenser. Nordmenn er jenter som er glad i jenter, gutter som er glad i gutter, og jenter, gutter som er glad i gutter, og jenter og gutter som er glad i hverandre. Nordmenn tror på Gud, Allah, altet og ingenting. Nordmenn liker Grieg, Hugo, Helbilis og Kari Bremnes. Med andre ord, Norge er dere. Norge er oss. Mitt største håp for Norge er at vi skal klare å ta vare på hverandre. At vi skal bygge dette landet videre på tillit, fellesskap og raushet.

## Bokmål

 ### Nordmenn er nordlendinger, trøndere, sørlendinger og folk fra alle andre regioner. Nordmenn er også innvandret fra Afghanistan, Pakistan og Polen, Sverige, Somalia og Syria. Det er ikke alltid så lett å si hvor vi er fra, hvilken nasjonalitet vi tilhører. Det vi kaller hjem er der hjertet vårt er, og det kan ikke alltid plasseres innenfor landegrenser. Nordmenn er jenter som er glad i jenter, gutter som er glad i gutter, og jenter og gutter som er glad i hverandre. Nordmenn tror på Gud, Allah, altet og ingenting. Nordmenn liker Grieg, Hugo, Helbilis og Kari Bremnes. Med andre ord, Norge er dere. Norge er oss. Mitt største håp for Norge er at vi skal klare å ta vare på hverandre. At vi skal bygge dette landet videre på tillid, fellesskap og raushet.

# Test med norsk dialekt, satt til nynorsk

In [14]:
result = pipe("sound/Språkrådets_årsrapport.mp3",  generate_kwargs={"language": "nynorsk"}, return_timestamps=True)
print(result["text"])

# return_timestamps=True)


Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


 Hei, eg heiter Åse Vetås, og eg er direktør i Språkrådet. Språkrådet er statens forvaltningsorgan i språkspørsmål, og vi fyller opp språkpolitiske mål og strategier på oppdrag fra Kultur- og Ligestillingsdepartementet. Nå er årsrapporten vår for 2022 klar, og eg vil gjerne gi deg nokon smagebeder av det arbeidet vi gjorde i året som gikk. Det øverordne målet for den norske språkpolitikken er å styrke norsk som fullverdikt språk og sikre språkmangfoldet i Norge. Hovedmålet for arbeidet vårt er å fremme statusen til å bruke norsk språk på alle samfunnsområder. Femme statusen til å bruke norsk språk på alle samfunnsområder. Det har vært styrande for de ressursprioriteringar vi har gjort i 2022, og det ligger også inne i de fire delmålene vi har for arbeidet vårt. Det første delmålet handler om retten innbyggerne i Norge har til å møte og bruke språket sitt, eller språket sine, enten det er nynorsk, bokmål, samiske språk, norsk tegnspråk, kvensk, romani eller romanes. Den nye språkloven b

# Test med norsk dialekt, satt til bokmål

In [15]:
result = pipe("sound/Språkrådets_årsrapport.mp3",  generate_kwargs={"language": "norwegian"}, return_timestamps=True)
print(result["text"])

# return_timestamps=True)


 Hei, jeg heter Åse Vetås og er direktør i Språkrådet. Språkrådet er statens forvaltningsorgan i språkspørsmål, og vi fyller opp språkpolitiske mål og strategier på oppdrag fra Kultur- og Ligestillingsdepartementet. Nå er årsrapporten vår for 2022 klar, og jeg vil gjerne gi deg noen smagebeder av det arbeidet vi gjorde i året som gikk. Det overordnede målet for den norske språkpolitikken er å styrke norsk som fullverdikt språk og sikre språkmangfoldet i Norge. Hovedmålet for arbeidet vårt er å fremme statusen til å bruke norsk språk på alle samfunnsområder. stemmer statusen til å bruke norsk språk på alle samfunnsområder. Det har vært styrende for de ressursprioriteringene vi har gjort i 2022, og det ligger også inne i de fire delmålene som vi har for arbeidet vårt. Det første delmålet handler om retten innbyggerne i Norge har til å møte og bruke språket sitt eller språket sine, til å møte og bruke språket sitt eller språket sine, enten det er nynorsk, bokmål, samiske språk, norsk tegn